In [ ]:
%matplotlib inline

## YOLO

In [ ]:
from src.hpe.common.cleanup import remove_temp_factory
from src.hpe.yolo.mutators import adapt_to_bgr
from src.hpe.yolo.performance import estimate_distances

In [ ]:
# default estimation
distances = estimate_distances()

In [ ]:
# estimation on bgr images
distances_bgr = estimate_distances(
    dataset_name="distances_bgr",
    image_path_mutators=[adapt_to_bgr],
    clean_up=[remove_temp_factory()]
)

In [ ]:
from src.hpe.yolo.performance import read_distances, estimate_performance
from src.hpe.yolo.plot import plot_yolo_average_distances

In [ ]:
estimate_performance(name="Yolov11x-pose")

In [ ]:
distances = read_distances()
plot_yolo_average_distances(distances=distances)

In [ ]:
# calculate and log performance on bgr images
estimate_performance(
    name="Yolov11x-pose on bgr",
    image_path_mutators=[adapt_to_bgr],
    clean_up=[remove_temp_factory()]
)

In [ ]:
distances_bgr = read_distances(dataset_name="distances_bgr")
plot_yolo_average_distances(distances=distances_bgr)

## MediaPipe

In [ ]:
from src.hpe.mp.performance import estimate_distances

In [ ]:
distances = estimate_distances()

In [ ]:
from cv2 import cvtColor, COLOR_RGB2BGR
from cv2.typing import MatLike

def convert_to_bgr(image: MatLike) -> MatLike:
    return cvtColor(image, COLOR_RGB2BGR)

In [ ]:
# estimation on bgr images
distances_bgr = estimate_distances(
    dataset_name="distances_bgr",
    image_mutators=[convert_to_bgr]
)

In [ ]:
from src.hpe.mp.performance import read_distances
from src.hpe.mp.plot import plot_mediapipe_average_distances

In [ ]:
distances = read_distances()
plot_mediapipe_average_distances(distances=distances)

In [ ]:
distances_bgr = read_distances(dataset_name="distances_bgr")
plot_mediapipe_average_distances(distances=distances_bgr)

## Comparison